In [8]:
import pyodbc
import os, json
import pandas as pd
import datetime
from datetime import datetime, timedelta

arquivo_json = os.path.join(os.environ['USERPROFILE'], "config_robos.json")

with open(arquivo_json, encoding='utf-8') as f:
    dados = json.load(f)

usuarioID = dados["senha_rede"]["chave"]
senhaAcesso = dados["senha_rede"]["senha"]




# Defina suas credenciais e propriedades de conexão
def new_func(usuarioID, senhaAcesso, strSQL):
    host = "vdados.petrobras.com.br"
    port = 9401
    username = usuarioID
    password = senhaAcesso
    database = "S10267"  # O dataSource

    connection = pyodbc.connect('dsn=TDVCPBIP')



    strSQL = strSQL
    df = pd.read_sql(strSQL, connection)

    return df



In [5]:
# Calcula a data formatada como hoje - 35 dias
dataFormatada = (datetime.now() - timedelta(days=35)).strftime('%Y-%m-%d')
strSQL = f"""
SELECT 
    ESQUEMA.OBJNR AS OBJETO_ORDEM, 
    STD.TXT04 AS STATUS, 
    STD.TXT30 AS TEXTO_STATUS, 
    ST.USNAM AS USUARIO_MODIFICACAO_STATUS, 
    CAST(ST.UDATE AS DATE) AS DATA_MODIFICACAO_STATUS, 
    CAST(ST.UTIME AS TIME) AS HORA_MODIFICACAO_STATUS, 
    ST.INACT AS INDICADOR_STATUS_INATIVO, 
    ST.CHIND AS INDICADOR_TIPO_MODIFICACAO, 
    ST._CHANGE_TIME AS DATA_HORA_ATUALIZACAO_ODS
FROM 
    DADOS_BRUTOS.K_PM_MANUTENCAO.JSTO AS ESQUEMA
INNER JOIN 
    DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST ON ESQUEMA.OBJNR = ST.OBJNR
INNER JOIN 
    DADOS_BRUTOS.K_PM_MANUTENCAO.TJ30T AS STD 
    ON ST.STAT = STD.ESTAT AND ESQUEMA.STSMA = STD.STSMA
INNER JOIN (
    SELECT DISTINCT O2.OBJNR
    FROM DADOS_BRUTOS.K_PM_MANUTENCAO.AUFK AS O2
    INNER JOIN DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST2 ON O2.OBJNR = ST2.OBJNR
    WHERE 
        O2.WERKS = '1400' AND 
        O2.AEDAT > '{dataFormatada}' AND 
        ST2.UDATE > '{dataFormatada}'
) AS FILTRADOS ON ESQUEMA.OBJNR = FILTRADOS.OBJNR
WHERE 
    STD.TXT04 IN ('FELE', 'FINT', 'FENG', 'FCAL', 'FMEC', 'DLEX', 'FAPL', 'CPXC', 'PTTC') AND 
    ST.UDATE > '{dataFormatada}'
"""

new_func(usuarioID, senhaAcesso, strSQL)

C:\Users\D25E\AppData\Local\Temp\ipykernel_28292\2787186065.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


,OBJETO_ORDEM,STATUS,TEXTO_STATUS,USUARIO_MODIFICACAO_STATUS,DATA_MODIFICACAO_STATUS,HORA_MODIFICACAO_STATUS,INDICADOR_STATUS_INATIVO,INDICADOR_TIPO_MODIFICACAO,DATA_HORA_ATUALIZACAO_ODS
0,OR002028117629,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-04,17:04:30,,I,2025-03-17 13:02:24.161
1,OR002029804488,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:55:53,,U,2025-03-17 13:02:24.171
2,OR002029834745,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:54:28,,U,2025-03-17 13:02:24.179
3,OR002029804486,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:15:06,,U,2025-03-17 13:02:24.179
4,OR002029835966,CPXC,Complexidade de planej Baixa,RXTS,2025-01-06,12:10:28,,U,2025-03-17 13:02:24.175
...,...,...,...,...,...,...,...,...,...
12548,OR002030452846,DLEX,Delineamento Executado,EBGK,2025-04-24,13:38:33,,I,2025-04-24 13:38:35.248
12549,OR002029936700,DLEX,Delineamento Executado,DQH1,2025-01-07,10:42:10,,I,2025-03-17 13:02:24.140
12550,OR002030459063,DLEX,Delineamento Executado,ASB5,2025-04-28,09:56:06,,I,2025-04-28 09:56:20.027
12551,OR002029719186,DLEX,Delineamento Executado,G0A8,2025-04-09,12:20:09,,I,2025-04-09 12:20:12.494


In [10]:
strSQL = f"""SELECT TOP 1000 * FROM WHERE AUFK = '1400' """

df = new_func(usuarioID, senhaAcesso, strSQL)

df

C:\Users\D25E\AppData\Local\Temp\ipykernel_37052\2787186065.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


DatabaseError: Execution failed on sql 'SELECT TOP 1000 * FROM WHERE AUFK = '1400' ': ('HY000', "[HY000] Invalid table 'WHERE'.  On line 1, column 15.  [parser-2906435] [Log ID: da71b3ea-9485-42ad-b4b6-62720c6254f3]  (-1) (SQLExecDirectW)")

In [17]:
import pandas as pd

# Lista de tabelas
'''tabelas = [
    "AFAB", "AFFH", "AFIH", "AFKO", "AFPO", "AFRU", "AFVC", "AFVU", "AFVV", "AUFK", "AUFM", "AUSP",
    "CRHD", "EQKT", "EQUI", "EQUZ", "H_T399I", "IFLOT", "IFLOTX", "ILOA", "IMEL", "IMEP", "IMPH",
    "IMPTT", "IMRG", "JCDS", "JEST", "JSTO", "MHIO", "MHIS", "MMPT", "MPLA", "MPOS", "OBJK", "PLAF",
    "PLAS", "PLKO", "PLKZ", "PLMZ", "PLPO", "PLWP", "STKO", "STPO",
    "T024I", "T351", "T351P", "T351X", "T353I", "T399G", "TC24", "TJ02", "TJ02T", "TJ30", "TJ30T"
]'''
#tabelas = ["VW_MEDIDA","VW_MEDIDA_ACAO","VW_MEDIDA_HIST_STATUS_SIS","VW_MEDIDA_HIST_STATUS_USU","VW_MEDIDA_ITEM_TEXTO_LONGO","VW_MEDIDA_STATUS_RT","VW_MEDIDA_TEXTO_LONGO","VW_NOTA","VW_NOTA_BUZIOS","VW_NOTA_CAUSA","VW_NOTA_HIST_STATUS_SIS","VW_NOTA_HIST_STATUS_USU","VW_NOTA_PARTE","VW_NOTA_STATUS","VW_NOTA_STATUS_MT","VW_NOTA_TEXTO_LONGO"]
tabelas = [
'QMEL',
'QMFE',
'QMIH',
'QMMA',
'QMSM',
'QMUR',
'QPCD',
'QPCT',
'QPGR']
valor_do_filtro_iwerk = "1400"
# Cria um objeto ExcelWriter para salvar os dados
output_file = "tabelas_relacao_Qtables.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Itera sobre as tabelas e executa a consulta para cada uma
    for tabela in tabelas:
        print(f"Processando tabela: {tabela}")
        strSQL = """
        SELECT TOP 100 MANDT, QMNUM, IWERK, ILOAN, ILOAI, EQUNR, BAUTL, EBORT, MSAUS, AUSVN, AUSBS, AUZTV, AUZTB, AUSZT, MAUEH, BTPLN, BEQUI, AUSWK, VERFV, VERFN, VERFM, ANLZV, ANLZN, ANLZE, INSPK, DATAN, INGRP, WARPL, ABNUM, WAPOS, KDAUF, KDPOS, SCREENTY, PLNTY, PLNNR, PLNAL, REVNR, LOC_DESC, ILART, NOTIFICATIONPERSONRESPONSIBLE, DATAMODELVERSION, MAINTNOTIFOVRLPROCPHASE, MAINTNOTIFOVRLPROCSUBPHASE, DUMMY_QMIH_INCL_EEW_PS, ZAEHL, LACD_DATE, OLD_LACD_DATE, PAMS_PTSTF, PAMS_PTSTT, PAMS_ATSTF, PAMS_ATSTT, PAMS_POOL, PAMS_KOSTL, PAMS_AUFNR, PAMS_PROID, PAMS_KOKRS, YYMANOBRA, YYVALVULASESTANQ, YYVALVULABYPASS, YYBYPASS_ECOS, YYPARADASISTEMA, YYTIPOBLOQUEIO, YYAREACLASSIF, YYAREANAOCLASSIF, YYSOBREMAR, YYRTA, YYGIM_FAM, YYAREA_RESPONS
FROM DADOS_BRUTOS.P_QM_QUALIDADE.QMIH;
            """
        
        
        
        #strSQL = f"SELECT TOP 1000 * FROM {tabela} WHERE IWERK = '{valor_do_filtro_mandt}'"
        
        try:
            df = new_func(usuarioID, senhaAcesso, strSQL)
            # Salva cada tabela em uma aba separada
            df.to_excel(writer, sheet_name=tabela[:31], index=False)  # Limita o nome da aba a 31 caracteres
        except Exception as e:
            print(f"Erro ao processar a tabela {tabela}: {e}")

print(f"Os dados foram salvos no arquivo {output_file}.")


Processando tabela: QMEL


C:\Users\D25E\AppData\Local\Temp\ipykernel_48100\3507244373.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


Processando tabela: QMFE
Processando tabela: QMIH
Processando tabela: QMMA
Processando tabela: QMSM
Processando tabela: QMUR
Processando tabela: QPCD
Processando tabela: QPCT
Processando tabela: QPGR
Os dados foram salvos no arquivo tabelas_relacao_Qtables.xlsx.
